<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Ejercicio de $Q$-learning. _Rock, paper, scissors, lizard, Spock_<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Grupo: NeoTech<br>Última actualización: 2023-05-06</small></i></div>

                                                  

***

## Introducción

En inteligencia artificial, el **aprendizaje por refuerzo** es una técnica de aprendizaje automático que permite que un agente aprenda a **tomar decisiones a través de la interacción con un entorno**. En este contexto, el algoritmo $Q$-learning es una técnica popular para el aprendizaje por refuerzo que permite que un agente aprenda a maximizar una recompensa a largo plazo al tomar decisiones óptimas en un entorno incierto.

En esta práctica, se explorará el uso del algoritmo Q-learning para entrenar a un agente para que juegue al juego _rock, paper, scissors, lizard, spock_ a partir de información extraída de un jugardor con ciertos sesgos, cosa que le sucede generalmente a los humanos.

## Objetivos

Vamos a desarrollar un agente para que aprenda a jugar al juego de manera relativamente competente jugando al juego indicado usando para ello $Q$-learning.

Primero desarrollaremos el algoritmo básico para aprender de unos datos extraídos de un jugador. Luego, se pedirá que se altere (si se desea) el agente para quu juegue contra los agentes deel resto de los estudiantes.

## Imports y configuración

A continuación importaremos las librerías que se usarán a lo largo del _notebook_.

In [ ]:
import enum
import typing
import urllib

import numpy as np

***

## Descripción del problema

Este juego es una extensión del clásico juego «piedra, papel o tijera» e incluye dos elementos adicionales, «lagarto» (_lizard_ en inglés) y «Spock» (extraterrestre de la raza Vulcana, conocido por su lógica y falta de emociones). El objetivo del juego es seleccionar una de las cinco opciones posibles y ganar a la opción elegida por el oponente, siguiendo las siguientes reglas:

- La piedra aplasta la tijera y aplasta al lagarto
- La tijera corta el papel y decapita al lagarto
- El papel envuelve la piedra y desautoriza a Spock
- El lagarto envenena a Spock y come el papel
- Spock rompe las tijeras y vaporiza la piedra

Comenzamos definiendo las opciones posibles del juego, que se corresponderán con las acciones posibles que puede realizar un agente en el entorno.

In [ ]:
class Action(enum.Enum):
    """Cada una de las posibles figuras."""
    ROCK = '🪨'
    PAPER = '🧻'
    SCISSORS = '✂️'
    LIZARD = '🦎'
    SPOCK = '🖖'

Los movimientos junto con sus recompensas (si no nos hemos equivocado) se representan en el siguiente diccionario. Las recompensas se establecen en 1, 0, -1 dependiendo de si la primera opción gana a, empata con o pierde frente a la segunda, respectivamente.

In [ ]:
MOVES_AND_REWARDS = {
    (Action.ROCK, Action.ROCK): 0, (Action.ROCK, Action.PAPER): -1,
    (Action.ROCK, Action.SCISSORS): 1, (Action.ROCK, Action.LIZARD): 1,
    (Action.ROCK, Action.SPOCK): -1,
    (Action.PAPER, Action.ROCK): 1, (Action.PAPER, Action.PAPER): 0,
    (Action.PAPER, Action.SCISSORS): -1, (Action.PAPER, Action.LIZARD): -1,
    (Action.PAPER, Action.SPOCK): 1,
    (Action.SCISSORS, Action.ROCK): -1, (Action.SCISSORS, Action.PAPER): 1,
    (Action.SCISSORS, Action.SCISSORS): 0, (Action.SCISSORS, Action.LIZARD): 1,
    (Action.SCISSORS, Action.SPOCK): -1,
    (Action.LIZARD, Action.ROCK): -1, (Action.LIZARD, Action.PAPER): 1,
    (Action.LIZARD, Action.SCISSORS): -1, (Action.LIZARD, Action.LIZARD): 0,
    (Action.LIZARD, Action.SPOCK): 1,
    (Action.SPOCK, Action.ROCK): 1, (Action.SPOCK, Action.PAPER): -1,
    (Action.SPOCK, Action.SCISSORS): 1, (Action.SPOCK, Action.LIZARD): -1,
    (Action.SPOCK, Action.SPOCK): 0,
}

Con esta información, crearemos el juego para que el agente (humano o máquina) pueda jugar.

In [ ]:
class Game:
    RENDER_MODE_HUMAN = 'human'
    
    def __init__(self, render_mode=None):
        self.render_mode = render_mode

    def play(self, p1_action, p2_action):
        result = MOVES_AND_REWARDS[(p1_action, p2_action)]
        if self.render_mode == 'human':
            self.render(p1_action, p2_action, result)
        return result
    
    @staticmethod
    def render(p1_action, p2_action, result):
        if result == 0:
            print(f'{p1_action.value} tie!')
        elif result == 1:
            print(f'{p1_action.value} beats {p2_action.value}')
        elif result == -1:
            print(f'{p2_action.value} beats {p1_action.value}')
        else:
            raise ValueError(f'{p1_action}, {p2_action}, {result}')

game = Game(render_mode='human')
game.play(np.random.choice(list(Action)), np.random.choice(list(Action)))

## Agente que aprende mediante $Q$-learning

Vamos a crear una clase auxiliar denominada `Transition` que va a guardar la información de una transición, definida por el estado origen, el estado destino, la acción por la que ocurrió dicha transición y la recompensa de la misma. De esta manera tendremos en un mismo objeto toda la información relacionada.

In [ ]:
class Transition(typing.NamedTuple):
    """Representa la transición de un estado al siguiente"""
    prev_state: int              # Estado origen de la transición
    next_state: int              # Estado destino de la transición
    action: Action               # Acción que provocó esta transición
    reward: typing.SupportsFloat # Recompensa obtenida

El estado lo representaremos como un entero ya que, en este problema en concreto, sólo tenemos un estado. Hemos preferido mantener aún así los atributos `prev_state` y `next_state` para que el código se corresponda con las fórmulas explicadas.

### Ejercicio 1. Definición del agente

Definiremos el agente que aprenderá a jugar a partir de las jugadas de los demás. Debe implementar los métodos indicados para que cumplan la firma y el comentario.

Se permite escribir código sólo entre el espacio delimitado por los comentarios `# START` y `# END`.

In [ ]:
import collections
import requests


# Hemos cambiado algún nombre por comodidad
class Agent:
    def __init__(self, name: str, q_table: typing.Any=None):
        """Inicializa este objeto.
        
        :param name: El nombre del agente, para identificarle.
        :param q_table: Una tabla q de valores. Es opcional.
        """
        self.name = name
        if q_table:
            self.q_table = q_table
        else:
            # Inicializa la Q-table a 0
            self.q_table = {b.value: 0.0 for b in Action}
            self.updated_table = self.q_table.copy()
            

    def decide(self, state:int, epsilon: typing.SupportsFloat=0) -> Action:
        """Decide la acción a ejecutar.
        
        :param state: El estado en el que nos encontramos.
        :param epsilon: Un valor entre 0 y 1 que representa, según la estrategia
            ε-greedy, la probabilidad de que la acción sea elegida de manera
            aleatoria de entre todas las opciones posibles. Es opcional, y si
            no se especifica su valor es 0 (sin probabilidades de que se elija
            una acción aleatoria).
        :param returns: La acción a ejecutar.
        """
        if np.random.random() < epsilon:
            # Con probabilidad epsilon, elegir una acción random
            return np.random.choice(list(Action))
        if state in self.q_table:
            # Si el estado está en la Q-table, elige el que tenga el valor más alto
            argmax = np.argmax(list(self.q_table[state].values()))
            return list(Action)[argmax]
        else:
            # Si no lo está, elige uno random
            self.q_table[state] = {action.value: 0.0 for action in Action}
            return np.random.choice(list(Action))
        

    def update(self, t: Transition, alpha=0.1, gamma=0.95):
        """Actualiza el estado interno de acuerdo a la experiencia vivida.
        
        :param transition: Toda la información correspondiente a la transición
            de la que queremos aprender.
        :param 𝛼: El factor de aprendizaje del cambio en el valor q. Por
            defecto es 0.1
        :param 𝛾: La influencia de la recompensa a largo plazo en el valor q a
            actualizar. Va de 0 (sin influencia) a 1 (misma influencia que el
            valor actual). Por defecto es 0.95.
        """
        if t.prev_state not in self.q_table:
            # Si el prev estado no está, añadirlo
            self.q_table[t.prev_state] = {action.value: 0.0 for action in Action}
        current_q = self.q_table[t.prev_state][t.action.value]
        next_q = max(list(self.q_table[t.next_state].values()))
        td_target = t.reward + gamma * next_q
        td_error = td_target - current_q
        new_q = current_q + alpha * td_error
        self.q_table[t.prev_state][t.action.value] = new_q
        self.updated_table = self.q_table.copy()

    def __str__(self) -> str:
        """Representación textual de la tabla Q del agente.
        
        :returns: Una cadena indicando la estructura interna de la tabla Q.
        """
        table_str = "Agent - Q Table:\n"
        for state, actions in self.q_table.items():
            table_str += f"State: {state}\n"
            if isinstance(actions, float):
                table_str += f"  {Action(state).name}: {actions}\n"
            else:
                for action, value in actions.items():
                    table_str += f"  {action}: {value}\n"
        return table_str

## Entrenamiento

Para el entrenamiento usaremos el dataset localizado en <https://blazaid.github.io/Aprendizaje-profundo/Datasets/rock-paper-scissors-lizard-spock.trn>, el cual contiene una secuencia de opciones que ha tomado un jugador en una partida ficticia de este juego. Comenzamos cargando el fichero:

In [ ]:
dataset_url = 'https://blazaid.github.io/Aprendizaje-profundo/Datasets/rock-paper-scissors-lizard-spock.trn'

player2_actions = []
with urllib.request.urlopen(dataset_url) as f:
    for line in f:
        move = line.decode('utf-8').strip().upper()
        if move:
            player2_actions.append(Action[move])

Con el dataset descargado, ya podemos realizar el entrenamiento.

In [ ]:
𝜀 = 1
𝛿𝜀 = 𝜀 / len(player2_actions)

game = Game()
agent = Agent(name='Agent')

state = 0  # El entorno (juego) no tiene estado, así que siempre será el mismo
for p2_action in player2_actions:
    p1_action = agent.decide(state, 𝛆)
    reward = game.play(p1_action, p2_action)

    # Actualizamos el agente
    agent.update(Transition(
        prev_state=state,
        next_state=state,
        action=p1_action,
        reward=reward
    ))

    # Actualizamos 𝜀
    𝜀 -= 𝛿𝜀 if 𝜀 > 0 else 0

Tras el entrenamiento, podemos ver cómo están de repartidos los valores de la tabla $Q$:

In [ ]:
print(agent)

Veamos qué tal se comporta el con un conjunto de datos que nunca ha visto del mismo contrincante.

In [ ]:
dataset_url = 'https://blazaid.github.io/Aprendizaje-profundo/Datasets/rock-paper-scissors-lizard-spock.tst'

player2_actions = []
with urllib.request.urlopen(dataset_url) as f:
    for line in f:
        move = line.decode('utf-8').strip().upper()
        if move:
            player2_actions.append(Action[move])

stats = collections.defaultdict(int)
state = 0
for p2_action in player2_actions:
    p1_action = agent.decide(state)
    reward = game.play(p1_action, p2_action)
    if reward == 1:
        stats['wins'] += 1
    elif reward == -1:
        stats['loses'] += 1
    else:
        stats['ties'] += 1

print(dict(stats))

Por lo visto ha aprendido un poco del sesgo que tenía el contrincante. ¡Bien por nuestro agente! Ya está un poco más cerca de dominar el mundo.

### Ejercicio 2. Nuevo agente para competición

Para hacer las cosas un poco más interesantes, vamos a hacer una pequeña competición. Se trata de desarrollar un agente como el que acabamos de hacer, pero donde los contrincantes serán el resto de agentes de los grupos. Se permite cualquier implementación, siempre y cuando:

- Herede de la clase `Agent` suministrada.
- El método `__init__` no admita ningún parámetro adicional.

En la siguiente celda se ofrece el esqueleto del agente que competirá. Como se puede ver:

- Tiene un nombre
- A la hora de decidir la acción recibirá un estado, que siempre será 0 y devolverá la opción a realizar.

In [ ]:
import abc

class Agent(metaclass=abc.ABCMeta):
    
    @abc.abstractmethod
    def __init__(self, name: str):
        """Inicializa el objeto.
        
        :param name: El nombre del agente.
        """
        self.name = name
        # START
        # END

    @abc.abstractmethod
    def decide(self, state:int) -> Action:
        """Decide la acción a llevar a cabo dado el estado actual.
        
        :param state: El estado en el que se encuentra el agente.
        :returns: La acción a llevar a cabo.
        """
        # START
        # END
    
    def update(self, transition: Transition):
        """Actualiza (si es necesario) el estado interno del agente.
        
        :param transition: La información de la transición efectuada.
        """
        pass
    
    def __str__(self):
        return self.name

Por ejemplo, un par de posibles implementaciones (que no usan nada de $q$-learning, y que no se recomiendan) son las siguientes:

In [ ]:
class Botnifacio(Agent):
    def __init__(self):
        super().__init__(name='Botnifacio')
    
    def decide(self, state:int) -> Action:
        return np.random.choice(list(Action))


class Gustabot(Agent):
    def __init__(self):
        super().__init__(name='Gustabot')
        self.weights = np.random.random(5)
        self.weights /= sum(self.weights)
    
    def decide(self, state:int) -> Action:
        return np.random.choice(list(Action), p=self.weights)
    
    def update(self, transition: Transition):
        self.weights = np.random.random(5)
        self.weights /= sum(self.weights)

El agente se deberá desarrollar en la siguiente celda, la cual incluirá todo lo necesario para instanciarlo y que funcione. Se recomienda la competición entre grupos antes de la entrega final.

In [ ]:
# Implementar aquí al agente

La competición será una liga entre los agentes de cada grupo a 100 partidas.

Antes de dichas 100 partidas, se hará un previo de 10000 partidas con el fin de que cada uno de los agentes "aprenda" a competir contra el contrario. Sólo en esta fase se invocará el método `update`.

La competición se realizará como sigue:

In [ ]:
import itertools

FRIENDLY_SETS = 10000
COMPETITION_SETS = 10

competitors = [
    Gustabot(),
    Botnifacio(),
]

leaderboard = {c: 0 for c in sorted(competitors, key=lambda x: x.__str__())}
game = Game()
for p1, p2 in itertools.combinations(leaderboard.keys(), 2):
    # Amistoso
    s = 0
    for i in range(FRIENDLY_SETS):
        a1 = p1.decide(s)
        a2 = p2.decide(s)
        reward = game.play(a1, a2)
        p1.update(Transition(prev_state=s, next_state=s, action=a1, reward=reward))
        p2.update(Transition(prev_state=s, next_state=s, action=a2, reward=-reward))
    
    # Competición
    s = 0
    r1 = r2 = 0
    for i in range(COMPETITION_SETS):
        a1 = p1.decide(s)
        a2 = p2.decide(s)
        reward = game.play(a1, a2)
        r1 += reward
        r2 -= reward

    # Actualización de marcadores globales
    leaderboard[p1] += r1
    leaderboard[p2] += r2
    
    print(f'{p1}: {r1}, {p2}: {r2}')
    

print('LEADERBOARD')
for c, r in sorted(leaderboard.items(), key=lambda t: t[1], reverse=True):
    print(f'{r:<10}\t{c}')

La calificación de cada grupo irá en función de la puntuación final que haya conseguido.

## Resumiendo

Hemos desarrollado un agente para un juego diferente que aprender a tomar decisiones óptimas en cada situación, a partir de la retroalimentación proporcionada por el entorno del juego. Hemos observado cómo el agente ha ido mejorando gradualmente su desempeño en el juego, a medida que ha acumulado experiencia y ha ajustado sus valores de Q.

Además, hemos podido comprobar cómo el agente es capaz de adaptarse a diferentes estrategias de juego de su oponente, lo que demuestra su capacidad para generalizar y tomar decisiones en situaciones nuevas.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>